# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [191]:
!pip install pathway bokeh panel --quiet

print("Hello")

Hello


In [192]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import numpy as np
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn
from bokeh.io import output_notebook, show

print("Hello")

Hello


In [193]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 1: Importing and Preprocessing the Data

In [194]:
# Load the dataset (replace with your actual file if needed)
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')

# Combine date and time columns if needed
if 'LastUpdatedDate' in df.columns and 'LastUpdatedTime' in df.columns:
    df['Timestamp'] = pd.to_datetime(
        df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
        format='%d-%m-%Y %H:%M:%S'
    )
else:
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Sort by time
df = df.sort_values('Timestamp').reset_index(drop=True)

# Ensure a unique lot identifier column (LotID)
if 'LotID' not in df.columns:
    df['LotID'] = df['Latitude'].astype(str) + '_' + df['Longitude'].astype(str)

# Save a streaming-ready CSV for Pathway
df[["Timestamp", "Occupancy", "Capacity", "LotID"]].to_csv("parking_stream_model2.csv", index=False)

print("Hello")
df.columns

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

Hello


Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'Timestamp',
       'LotID'],
      dtype='object')

In [195]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    LotID: str
    Price_Model2: float

In [196]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv(
    "parking_stream_model2.csv",
    schema=ParkingSchema,
    input_rate=1000  # Adjust as needed for speed
)

In [197]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


print("Hello")

Hello


In [198]:
# Clone your repo (first time)
!git clone https://github.com/rahmani143/IITG_CA_AIML.git

# After editing, commit and push (replace with your info)
!cd IITG_CA_AIML && git add . && git commit -m "Update from Colab" && git push


fatal: destination path 'IITG_CA_AIML' already exists and is not an empty directory.
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@c1774eaf447d.(none)')


# Step 2: Making a simple pricing function

In [199]:


cols_to_numeric = ['Occupancy', 'Capacity', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay']
for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Debug: Print rows with NaN or zero in required columns
for col in cols_to_numeric:
    nan_rows = df[df[col].isna()]
    if not nan_rows.empty:
        print(f"DEBUG: Rows with NaN in {col}:")
        print(nan_rows[['Timestamp', 'LotID', col]])
zero_cap = df[df['Capacity'] == 0]
if not zero_cap.empty:
    print("DEBUG: Rows with zero Capacity:")
    print(zero_cap[['Timestamp', 'LotID', 'Capacity']])

# Drop rows with NaN or zero in required columns
df = df.dropna(subset=cols_to_numeric)
df = df[df['Capacity'] != 0]

vehicle_type_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_type_weights).fillna(1.0)

df['Norm_Occupancy'] = df['Occupancy'] / df['Capacity']
df['Norm_Queue'] = df['QueueLength'] / df['QueueLength'].max()
df['Norm_Traffic'] = df['TrafficConditionNearby'] / df['TrafficConditionNearby'].max()

ALPHA = 1.0
BETA = 0.5
GAMMA = 0.3
DELTA = 0.8
EPSILON = 0.7

df['Raw_Demand'] = (
    ALPHA * df['Norm_Occupancy'] +
    BETA * df['Norm_Queue'] -
    GAMMA * df['Norm_Traffic'] +
    DELTA * df['IsSpecialDay'] +
    EPSILON * df['VehicleTypeWeight']
)

if df['Raw_Demand'].max() != df['Raw_Demand'].min():
    df['Norm_Demand'] = (df['Raw_Demand'] - df['Raw_Demand'].min()) / (df['Raw_Demand'].max() - df['Raw_Demand'].min())
else:
    df['Norm_Demand'] = 0.0

BASE_PRICE = 10.0
LAMBDA = 1.0
df['Price_Model2'] = BASE_PRICE * (1 + LAMBDA * df['Norm_Demand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=BASE_PRICE*0.5, upper=BASE_PRICE*2)

# Final debug: Print and drop any rows with invalid Price_Model2
df['Price_Model2'] = pd.to_numeric(df['Price_Model2'], errors='coerce')
bad_rows = df[df['Price_Model2'].isna()]
if not bad_rows.empty:
    print("DEBUG: Dropping rows with invalid Price_Model2 before saving for Pathway:")
    print(bad_rows[['Timestamp', 'LotID', 'Price_Model2']])
df = df[df['Price_Model2'].notna()]

df[["Timestamp", "Occupancy", "Capacity", "LotID", "Price_Model2"]].to_csv("parking_stream_model2.csv", index=False)
print("Model 2 demand-based pricing calculated, cleaned, and saved.")


DEBUG: Rows with NaN in TrafficConditionNearby:
                Timestamp                    LotID  TrafficConditionNearby
0     2016-10-04 07:59:00  26.14453614_91.73617216                     NaN
1     2016-10-04 07:59:00  26.14001386_91.73099967                     NaN
2     2016-10-04 07:59:00   20.0000347_78.00000286                     NaN
3     2016-10-04 07:59:00  26.14004753_91.73097233                     NaN
4     2016-10-04 07:59:00  26.15050395_91.73353109                     NaN
...                   ...                      ...                     ...
18363 2016-12-19 16:30:00   26.14901995_91.7395035                     NaN
18364 2016-12-19 16:30:00  26.14449459_91.73620513                     NaN
18365 2016-12-19 16:30:00  26.14453614_91.73617216                     NaN
18366 2016-12-19 16:30:00  26.14749998_91.72797778                     NaN
18367 2016-12-19 16:30:00  26.15050395_91.73353109                     NaN

[18368 rows x 3 columns]
Model 2 demand-based prici

In [200]:

# ---- Clean and map TrafficConditionNearby ----
def extract_traffic(val):
    if isinstance(val, str):
        match = re.search(r'(low|average|high)', val)
        if match:
            return match.group(1)
        else:
            return None
    return val

traffic_map = {'low': 0, 'average': 1, 'high': 2}
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].apply(extract_traffic)
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].map(traffic_map)
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].fillna(0)

# ---- Ensure all required columns are numeric and fill missing values ----
BASE_PRICE = 10.0
required_cols = ['Timestamp', 'Occupancy', 'Capacity', 'LotID', 'Price_Model2']

df['Capacity'] = pd.to_numeric(df['Capacity'], errors='coerce').fillna(1)
df['Occupancy'] = pd.to_numeric(df['Occupancy'], errors='coerce').fillna(0)
df['QueueLength'] = pd.to_numeric(df['QueueLength'], errors='coerce').fillna(0)
df['TrafficConditionNearby'] = pd.to_numeric(df['TrafficConditionNearby'], errors='coerce').fillna(0)
df['IsSpecialDay'] = pd.to_numeric(df['IsSpecialDay'], errors='coerce').fillna(0)

# ---- Vehicle type weights ----
vehicle_type_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_type_weights).fillna(1.0)

# ---- Feature engineering ----
df['Norm_Occupancy'] = df['Occupancy'] / df['Capacity']
df['Norm_Queue'] = df['QueueLength'] / (df['QueueLength'].max() if df['QueueLength'].max() != 0 else 1)
df['Norm_Traffic'] = df['TrafficConditionNearby'] / (df['TrafficConditionNearby'].max() if df['TrafficConditionNearby'].max() != 0 else 1)

# ---- Demand and price calculation ----
ALPHA = 1.0
BETA = 0.5
GAMMA = 0.3
DELTA = 0.8
EPSILON = 0.7

df['Raw_Demand'] = (
    ALPHA * df['Norm_Occupancy'] +
    BETA * df['Norm_Queue'] -
    GAMMA * df['Norm_Traffic'] +
    DELTA * df['IsSpecialDay'] +
    EPSILON * df['VehicleTypeWeight']
)

if df['Raw_Demand'].max() != df['Raw_Demand'].min():
    df['Norm_Demand'] = (df['Raw_Demand'] - df['Raw_Demand'].min()) / (df['Raw_Demand'].max() - df['Raw_Demand'].min())
else:
    df['Norm_Demand'] = 0.0

LAMBDA = 1.0
df['Price_Model2'] = BASE_PRICE * (1 + LAMBDA * df['Norm_Demand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=BASE_PRICE*0.5, upper=BASE_PRICE*2)

# ---- Final cleaning: ensure no NaN/None in required columns ----
df['Price_Model2'] = pd.to_numeric(df['Price_Model2'], errors='coerce').fillna(BASE_PRICE)
df['Capacity'] = pd.to_numeric(df['Capacity'], errors='coerce').fillna(1)
df['Occupancy'] = pd.to_numeric(df['Occupancy'], errors='coerce').fillna(0)
df['LotID'] = df['LotID'].fillna('unknown')
df['Timestamp'] = df['Timestamp'].fillna('1970-01-01 00:00:00')

# Drop any row that still has NaN in any required column (should be none now)
df = df.dropna(subset=required_cols)

# ---- Save for Pathway ----
df[required_cols].to_csv("parking_stream_model2.csv", index=False)
print("Cleaned CSV saved for Pathway. No NaN or None in required columns.")


Cleaned CSV saved for Pathway. No NaN or None in required columns.




# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [203]:
# # # Activate the Panel extension to enable interactive visualizations
# # pn.extension()

# # Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
# def price_plotter(source):
#     # Create a Bokeh figure with datetime x-axis
#     fig = bokeh.plotting.figure(
#         height=400,
#         width=800,
#         title="Pathway: Daily Parking Price",
#         x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
#     )
#     # Plot a line graph showing how the price evolves over time
#     fig.line("t", "price", source=source, line_width=2, color="navy")

#     # Overlay red circles at each data point for better visibility
#     fig.circle("t", "price", source=source, size=6, color="red")

#     return fig

# # Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# # - 'price_plotter' is the rendering function
# # - 'sorting_col="t"' ensures the data is plotted in time order
# viz = delta_window.plot(price_plotter, sorting_col="t")

# # Create a Panel layout and make it servable as a web app
# # This line enables the interactive plot to be displayed when the app is served
# pn.Column(viz).servable()
# print("Hello")



# edit 0:



# import panel as pn
# import bokeh.plotting

# pn.extension('bokeh')

# def price_plotter(source):
#     fig = bokeh.plotting.figure(
#         height=400,
#         width=800,
#         title="Model 2: Demand-Based Price per Lot (Real-Time)",
#         x_axis_type="datetime",
#     )
#     lots = list(set(source.data['LotID']))
#     colors = bokeh.palettes.Category10[10] + bokeh.palettes.Category20[20]
#     legend_added = False  # Track if at least one legend is added
#     for i, lot in enumerate(lots):
#         mask = [l == lot for l in source.data['LotID']]
#         # Only plot if this lot has data
#         if any(mask):
#             fig.line(
#                 [t for t, m in zip(source.data['t'], mask) if m],
#                 [p for p, m in zip(source.data['Price_Model2'], mask) if m],
#                 line_width=2,
#                 color=colors[i % len(colors)],
#                 legend_label=f"Lot {lot}"
#             )
#             legend_added = True
#     if legend_added:
#         fig.legend.location = "top_left"
#     fig.xaxis.axis_label = 'Time'
#     fig.yaxis.axis_label = 'Price ($)'
#     return fig

# viz = data_with_time.plot(price_plotter, sorting_col="t")
# pn.Column(viz).servable()


# edit 1:


# from bokeh.plotting import figure, show, output_notebook
# import bokeh.palettes

# output_notebook()

# def plot_model2(df):
#     fig = figure(
#         height=400,
#         width=800,
#         title="Model 2: Demand-Based Price per Lot",
#         x_axis_type="datetime",
#     )
#     lots = df['LotID'].unique()
#     palette = bokeh.palettes.Category10[10] + bokeh.palettes.Category20[20]
#     for i, lot in enumerate(lots):
#         lot_df = df[df['LotID'] == lot]
#         fig.line(
#             lot_df['Timestamp'],
#             lot_df['Price_Model2'],
#             line_width=2,
#             color=palette[i % len(palette)],
#             legend_label=f"Lot {lot}"
#         )
#     fig.legend.location = "top_left"
#     fig.xaxis.axis_label = 'Time'
#     fig.yaxis.axis_label = 'Price ($)'
#     show(fig)

# plot_model2(df)



# edit 2:


from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import pandas as pd

output_notebook()

# Example: Load your data (replace with your own DataFrame)
# df = pd.read_csv("parking_stream_model2.csv")
# For demonstration, let's create a dummy DataFrame:
# df = pd.DataFrame({
#     "t": pd.date_range("2024-07-01", periods=10, freq="H"),
#     "LotID": ["A"]*5 + ["B"]*5,
#     "Price_Model2": [10,12,11,13,12,10,11,12,13,14]
# })

# Replace the above with your actual DataFrame, e.g.:
# df = data_with_time.to_pandas()  # If using Pathway's output

def price_plotter(df):
    fig = figure(
        height=400,
        width=800,
        title="Model 2: Demand-Based Price per Lot (Real-Time)",
        x_axis_type="datetime",
    )
    lots = df["LotID"].unique()
    colors = (list(getattr(__import__("bokeh.palettes"), "Category10")[10]) +
              list(getattr(__import__("bokeh.palettes"), "Category20")[20]))
    for i, lot in enumerate(lots):
        lot_df = df[df["LotID"] == lot]
        fig.line(
            lot_df["t"],
            lot_df["Price_Model2"],
            line_width=2,
            color=colors[i % len(colors)],
            legend_label=f"Lot {lot}"
        )
    fig.legend.location = "top_left"
    fig.xaxis.axis_label = 'Time'
    fig.yaxis.axis_label = 'Price ($)'
    show(fig)

# Example usage:
# price_plotter(df)


In [204]:


# # Start the Pathway pipeline execution in the background
# # - This triggers the real-time data stream processing defined above
# # - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()
print("Hello")



Output()